# Seattle to Topeka (Multi-location SC)

This is a rather famous multi-location supply chain.  

There are a two sources and three sinks. 
The goal is to procure a commodity ($r$) in any of the source, transport it and dispatch it in (or from) the sink locations.

The data is as shown: 


| Plants     | New York | Chicago | Topeka | Supply |
|-------------|-----------|----------|---------|---------|
| Seattle     | 2.5       | 1.7      | 1.8     | 350     |
| San Diego   | 2.5       | 1.8      | 1.4     | 600     |
| **Demand**  | 325       | 300      | 275     |         |

## Initialize the Model

The locations can be conveniently generated using .declare().
The US Dollar will be used as currency (a measure of economic impact)

In [1]:
from energia import *
from itertools import product

m = Model()
m.declare(Location, ['seattle', 'sandiego', 'newyork', 'chicago', 'topeka'])
m.usd = Currency()

## Resources 

There are three resources we consider. 

- $r\_consume$ - resource pre-procurement 
- $r$ - The insitu resource being transported 
- $r\_release$ - resource post-dispatch 

### Consumption Upper Bounds
Set the maximum consumption allowed at source locations

In [2]:
m.r_consume = Resource()
m.r_consume.consume(m.seattle) <= 350
m.r_consume.consume(m.sandiego) <= 600

⚖  Initiated r_consume balance in (seattle, t0)                             ⏱ 0.0001 s
🔗  Bound [≤] r_consume consume in (seattle, t0)                             ⏱ 0.0008 s
⚖  Initiated r_consume balance in (sandiego, t0)                            ⏱ 0.0001 s
🔗  Bound [≤] r_consume consume in (sandiego, t0)                            ⏱ 0.0007 s


### Release Lower Bounds
Set the minimum release allowed at sink locations

In [3]:
m.r_release = Resource()
m.r_release.release(m.newyork) >= 325
m.r_release.release(m.chicago) >= 300
m.r_release.release(m.topeka) >= 275

⚖  Initiated r_release balance in (newyork, t0)                             ⏱ 0.0001 s
🔗  Bound [≥] r_release release in (newyork, t0)                             ⏱ 0.0009 s
⚖  Initiated r_release balance in (chicago, t0)                             ⏱ 0.0001 s
🔗  Bound [≥] r_release release in (chicago, t0)                             ⏱ 0.0006 s
⚖  Initiated r_release balance in (topeka, t0)                              ⏱ 0.0001 s
🔗  Bound [≥] r_release release in (topeka, t0)                              ⏱ 0.0006 s


### Insitu Resource 

This is the primary resource being transported. The other two resources are dummy resources created for convenience 

In [4]:
m.r = Resource()

## Processes 

We create two dummy processes:

1. Purchase - which expends $r\_consume$ to produce $r$
2. Dispatch - which expends $r$ to produce $r\_release$

These are located at the sources and sinks respectively

In [5]:
m.purchase = Process()
m.purchase(m.r) == -m.r_consume
m.purchase.operate == True

m.dispatch = Process()
m.dispatch(-m.r) == m.r_release
m.dispatch.operate == True

m.purchase.locate(m.seattle, m.sandiego)
m.dispatch.locate(m.newyork, m.chicago, m.topeka)

💡  Assumed purchase capacity unbounded in (seattle, t0)                     ⏱ 0.0001 s
🧭  Mapped space for operate (purchase, seattle, t0) ⟺ (purchase, ntw, t0)   ⏱ 0.0001 s
🔗  Bound [≤] purchase operate in (seattle, t0)                              ⏱ 0.0007 s
💡  Assumed purchase operate bounded by capacity in (seattle, t0)            ⏱ 0.0011 s
💡  Assumed purchase capacity unbounded in (sandiego, t0)                    ⏱ 0.0001 s
🧭  Mapped space for operate (purchase, sandiego, t0) ⟺ (purchase, ntw, t0)  ⏱ 0.0001 s
🔗  Bound [≤] purchase operate in (sandiego, t0)                             ⏱ 0.0006 s
💡  Assumed purchase operate bounded by capacity in (sandiego, t0)           ⏱ 0.0010 s
⚖  Initiated r balance in (seattle, t0)                                     ⏱ 0.0001 s
🔗  Bound [=] r produce in (seattle, t0)                                     ⏱ 0.0005 s
⚖  Updated r_consume balance with expend(r_consume, seattle, t0, operate, purchase) ⏱ 0.0001 s
🔗  Bound [=] r_consume expend in (s

## Linking Locations 

Since the linkages between sources and sinks are unique. We can use the .Link() method.
A unique linkage between two locations can simply be accessed using: source - sink

For multiple linkages between two given locations. Named Linkage objects are needed. 


In [6]:
dist_dict = {
    m.seattle: {m.newyork: 2.5, m.chicago: 1.7, m.topeka: 1.8},
    m.sandiego: {m.newyork: 2.5, m.chicago: 1.8, m.topeka: 1.4},
}


for i, j in product([m.seattle, m.sandiego], [m.newyork, m.chicago, m.topeka]):
    m.Link(i, j, dist=dist_dict[i][j])

You can always check the linking

In [10]:
m.seattle.links(m.newyork)  # Link from Seattle to New York

seattle is source and newyork is sink in seattle-newyork


[seattle-newyork]

In [11]:
m.sandiego.connected(m.newyork)

True

## Transportation

In this mickey-mouse problem there are no dependent resources (produced and expended) besides the primary resource being transported.

A constant cost of 90 $\frac{\$}{\text{unit distance}}$ is considered

In [ ]:
m.channel = Transport()
m.channel(m.r) == 1.0  # 100% efficient

for i in dist_dict:
    for j in dist_dict[i]:
        m.usd.spend(m.channel.operate, i - j) == 90
        m.channel.locate(i - j)

## The Formulation

In [ ]:
m.show(True)

## Optimize!

The model can now be optimized

In [ ]:
m.usd.spend.opt()

## Solution

The solution pertaining to each aspect can be accessed individually. 

For the whole solution, use Model.output()

Seattle serves New York and Chicago

Sandiego serves New York and Topeka 

In [ ]:
m.ship_in.output()

Exports ($\mathbf{expt}$) and Imports ($\mathbf{impt}$) should match!

In [ ]:
m.ship_out.output()

Contribution to overall cost can be ascertained 

In [ ]:
m.spend.output()

### Solution as Dictionary 

In [ ]:
m.solution.asdict()